In [20]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [21]:

data = pd.read_csv('/mnt/ufs18/home-053/hejaseba/IMDBDataset.csv')

data['review'] = data['review'].str.lower()

In [22]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]
             

In [23]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [24]:
reviews = data_without_stopwords['clean_review']

reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])
 
sentiment = data_without_stopwords['sentiment']

In [25]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))


In [26]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)


In [27]:
tokenizer = Tokenizer(num_words=105124)
tokenizer.fit_on_texts(X_train+X_test)
words_to_index = tokenizer.word_index

In [28]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map
    

In [29]:
word_to_vec_map = read_glove_vector('/mnt/ufs18/home-053/hejaseba/glove.6B.300d.txt')


In [30]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len)) # (95419,50)

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None  and index!=105124:
    emb_matrix[index, :] = embedding_vector


In [31]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [32]:
import nltk, re, pprint, string
from nltk import word_tokenize, sent_tokenize
from nltk.util import ngrams
import random
import torch
from torch import nn


emb_ = torch.tensor(emb_matrix,requires_grad=True,dtype=torch.float64,device="cuda") # ,requires_grad=True


In [33]:
X_train_2 = []
max_ = 1505 # then try 600 
for i in range(0,len(X_train)):
    seq_ = word_tokenize(X_train[i])
    X_train_2.append([])
    for k in range(0,len(seq_)):
        ind_ = 0
        if(seq_[k] in words_to_index.keys()):
            ind_ = words_to_index[seq_[k]]
            if(ind_>=105124):
                ind_ = 0
        X_train_2[i].append(ind_)
    
    if(len(X_train_2[i])<max_):
        
        for j in range(0,max_-len(X_train_2[i])):
            X_train_2[i].append(0)

In [34]:
X_test_2 = []
max_ = 1505 # then try 600 
for i in range(0,len(X_test)):
    seq_ = word_tokenize(X_test[i])
    X_test_2.append([])
    for k in range(0,len(seq_)):
        ind_ = 0
        if(seq_[k] in words_to_index.keys()):
            ind_ = words_to_index[seq_[k]]
            if(ind_>=105124):
                ind_ = 0
        X_test_2[i].append(ind_)
    
    if(len(X_test_2[i])<max_):
        
        for j in range(0,max_-len(X_test_2[i])):
            X_test_2[i].append(0)

In [35]:
y_ =torch.tensor(Y_test,device="cuda")
y_ = torch.reshape(y_,(1,10000)).float() # X

y_ = y_[0]


y =torch.tensor(Y_train,device="cuda")
y = torch.reshape(y,(1,40000)).float() # X
y = y[0]

W1 = torch.normal(0,0.01, size=(300,512),requires_grad=True,device="cuda")
U1 = torch.normal(0,0.01, size=(1,512),requires_grad=True,device="cuda")
bw1 = torch.zeros(size=(1,512),requires_grad=True,device="cuda")


W2 = torch.normal(0,0.01, size=(512,512),requires_grad=True,device="cuda")
U2 = torch.normal(0,0.01, size=(1,512),requires_grad=True,device="cuda")
bw2 = torch.zeros(size=(1,512),requires_grad=True,device="cuda")

W3 = torch.normal(0,0.01, size=(512,512),requires_grad=True,device="cuda")
U3 = torch.normal(0,0.01, size=(1,512),requires_grad=True,device="cuda")
bw3 = torch.zeros(size=(1,512),requires_grad=True,device="cuda")


W4 = torch.normal(0,0.01, size=(512,512),requires_grad=True,device="cuda")
U4 = torch.normal(0,0.01, size=(1,512),requires_grad=True,device="cuda")
bw4 = torch.zeros(size=(1,512),requires_grad=True,device="cuda")



V = torch.normal(0,0.01, size=(512,2),requires_grad=True,device="cuda") # 3000 * 1
bv = torch.zeros(size=(1,2),requires_grad=True,device="cuda")

params = [W1,U1,bw1,W2,U2,bw2,W3,U3,bw3,W4,U4,bw4,V,bv,emb_] #,emb_]




In [36]:

r_ = nn.ReLU().cuda()

tan_ = nn.Tanh().cuda()
m = nn.Sigmoid().cuda()
s_ = nn.Softmax(dim=1) # (1,2)

def models(X):     
        h1 = torch.zeros(size=(1,512),device="cuda")
        h2 = torch.zeros(size=(1,512),device="cuda")
        h3 = torch.zeros(size=(1,512),device="cuda")
        h4 = torch.zeros(size=(1,512),device="cuda")

        for i in range(0,len(X[0])): 
              ind = [X[s_][i] for s_ in range(0,1) ]                
              if(0 not in ind):

                    
                h1 = ( emb_[ind].float() @ W1) + bw1 + (h1 * U1)
                h1 = r_(h1)
                
                h2 = ( h1 @ W2) + bw2 + (h2 * U2)
                h2 = r_(h2)

                h3 = ( h2 @ W3) + bw3 + (h3 * U3)
                h3 = r_(h3)       
                
                h4 = ( h3 @ W4) + bw4 + (h4 * U4)
                h4 = r_(h4)   
  
                
        y = h4 @ V + bv
        
        
        y = s_(y)
        return y
    
    

def cross_entropy(y_hat,y):
    to_ =  -torch.log(y_hat[range(len(y_hat)),y]+1e-8)
    to_ =  to_.sum(0,keepdim=True)
    to_ = to_ / 1.0
    return to_ + (((r_(torch.abs(U1) -1))**2).sum())/2 + (((r_(torch.abs(U2) -1))**2).sum())/2 + (((r_(torch.abs(U3) -1))**2).sum())/2 + (((r_(torch.abs(U4) -1))**2).sum())/2
    

trainer = torch.optim.Adam(params, lr=0.001)

In [37]:
def test(X,pass_=1):  
        h1 = torch.zeros(size=(1,512),device="cuda")
        h2 = torch.zeros(size=(1,512),device="cuda")
        h3 = torch.zeros(size=(1,512),device="cuda")
        h4 = torch.zeros(size=(1,512),device="cuda")

        if(pass_==11):
            len_=350
        else:
            len_ = len(X)
            
        
        for i in range(0,len_): # maybe take average
              ind = X[i]
              if(ind!=0):

                h1 = ( emb_[ind].float() @ W1) + bw1 + (h1 * U1)
                h1 = r_(h1)
                
                h2 = ( h1 @ W2) + bw2 + (h2 * U2)
                h2 = r_(h2)
                
                h3 = ( h2 @ W3) + bw3 + (h3 * U3)
                h3 = r_(h3)
                
                h4 = ( h3 @ W4) + bw4 + (h4 * U4)
                h4 = r_(h4)
                
        y = h4 @ V + bv
        
        
        y = s_(y)
        return y

In [46]:
trainer = torch.optim.Adam(params, lr=0.001)

import math
for k in range(0,10000):
    if(k>13):
        trainer = torch.optim.Adam(params, lr=0.0001)
    print(k)
    for i in range(0,1000):
            #print(i)
            j= random.randrange(1,39990,1)
            y_hat = models(X_train_2[j:j+1])
            l_ = cross_entropy(y_hat, y[j:j+1].long()) 
            
            #l_ = cross_entropy(y_hat.t(), y[j].long())            
            trainer.zero_grad()
            
            l_.backward()
            
            #grad_clipping(params, 1.0)
            trainer.step()

            #print(emb_.grad[0:10])
    # Predicting on Training data
    '''
    preds = []
    for i in range(0,300):
       #print(i)
       y_hat = test(X_train_2[i])
       preds.append(torch.argmax(y_hat).cpu())
    from sklearn.metrics import accuracy_score
    print("training accuracy ", accuracy_score(y[0:300].cpu().numpy(), preds))
    '''
    preds = []
    for i in range(0,300):
       #print(i)
       y_hat = models(X_test_2[i:i+1])
       preds.append(torch.argmax(y_hat).cpu())
    from sklearn.metrics import accuracy_score
    print("validation accuracy ", accuracy_score(y_[0:300].cpu().numpy(), preds))

0
validation accuracy  0.9066666666666666
1
validation accuracy  0.91
2
validation accuracy  0.9133333333333333
3
validation accuracy  0.9033333333333333
4
validation accuracy  0.9133333333333333
5


KeyboardInterrupt: 

In [49]:
preds = []
for i in range(300,10000):
       #print(i)
       y_hat = models(X_test_2[i:i+1])
       preds.append(torch.argmax(y_hat).cpu())
from sklearn.metrics import accuracy_score
print("test accuracy ", accuracy_score(y_[300:10000].cpu().numpy(), preds))

test accuracy  0.93
